<a href="https://colab.research.google.com/github/ashfaqfardin/NLP/blob/main/Text_Vectorizer_and_Word_Embeddings_Solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2. Text Vectorization in TensorFlow

In [1]:
import tensorflow as tf
import pandas as pd
import tensorflow_datasets as tfds

# Loading training data
imdb_data = tfds.load(name="imdb_reviews", split="train")
imdb_df = tfds.as_dataframe(imdb_data)
imdb_df['text'] = imdb_df['text'].str.decode('utf-8')
imdb_sample = imdb_df.sample(frac=0.2, random_state=100)

X_train = imdb_sample['text']
y_train = imdb_sample['label']

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteTB3FSC/imdb_reviews-train.tfrecord…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteTB3FSC/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteTB3FSC/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [2]:
# Loading testing data
imdb_test = tfds.load(name="imdb_reviews", split="test")
imdb_test_df = tfds.as_dataframe(imdb_test)
imdb_test_df['text'] = imdb_test_df['text'].str.decode('utf-8')
imdb_test_sample = imdb_test_df.sample(frac=0.2, random_state=100)

X_test = imdb_test_sample['text']
y_test = imdb_test_sample['label']

In [3]:
# Target distribution
imdb_test_sample['label'].value_counts()

1    2506
0    2494
Name: label, dtype: int64

In [4]:
# Check for missing values
imdb_test_sample.isna().sum()

label    0
text     0
dtype: int64

# 3. Creating the TextVectorizer Layer

In [5]:
from tensorflow.keras.layers import TextVectorization

max_tokens = 25000
output_sequence_length = 256

vectorizer_layer = TextVectorization(max_tokens=max_tokens,
                                     output_mode='int',
                                     standardize='lower',
                                     output_sequence_length=output_sequence_length)

vectorizer_layer.adapt(X_train)

# 4. Building the Model with the Vectorized Layer

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input, layers

model = Sequential()
model.add(Input(shape=(1,), dtype=tf.string))
model.add(vectorizer_layer)
model.add(layers.Dense(16))
model.add(layers.Dense(1, activation='sigmoid'))

opt = tf.optimizers.Adam(learning_rate=0.01)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10)
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test set accuracy: {test_acc}")

Epoch 1/10
157/157 [==============================] - 2s 6ms/step - loss: 712.6382 - accuracy: 0.5084
Epoch 2/10
157/157 [==============================] - 1s 5ms/step - loss: 34.1215 - accuracy: 0.5008
Epoch 3/10
157/157 [==============================] - 1s 5ms/step - loss: 34.6257 - accuracy: 0.5076
Epoch 4/10
157/157 [==============================] - 1s 5ms/step - loss: 24.9404 - accuracy: 0.5096
Epoch 5/10
157/157 [==============================] - 1s 5ms/step - loss: 30.1987 - accuracy: 0.5050
Epoch 6/10
157/157 [==============================] - 1s 5ms/step - loss: 21.6362 - accuracy: 0.5118
Epoch 7/10
157/157 [==============================] - 1s 5ms/step - loss: 22.9224 - accuracy: 0.5150
Epoch 8/10
157/157 [==============================] - 1s 5ms/step - loss: 16.1935 - accuracy: 0.5162
Epoch 9/10
157/157 [==============================] - 1s 5ms/step - loss: 16.1026 - accuracy: 0.5078
Epoch 10/10
157/157 [==============================] - 1s 4ms/step - loss: 12.5397 - accur

# 5. Create an Embedding Layer

In [7]:
output_dim = 128

from tensorflow.keras.layers import Embedding

embedding_layer = Embedding(input_dim=max_tokens,
                            output_dim=output_dim,
                            input_length=output_sequence_length)

# 6. Text Classification Model with Embedding Layer

In [8]:
vectorizer_layer = TextVectorization(max_tokens=max_tokens,
                                     output_mode='int',
                                     standardize='lower_and_strip_punctuation',
                                     ngrams=(1, 2),
                                     output_sequence_length=output_sequence_length)

vectorizer_layer.adapt(X_train)
embedding_layer = Embedding(input_dim=max_tokens,
                            output_dim=output_dim,
                            input_length=output_sequence_length)

model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(1,), dtype=tf.string))
model.add(vectorizer_layer)
model.add(embedding_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

opt = tf.optimizers.Adam(learning_rate=0.01)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10)
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test set accuracy: {test_acc}")

Epoch 1/10
157/157 [==============================] - 12s 65ms/step - loss: 0.6878 - accuracy: 0.5282
Epoch 2/10
157/157 [==============================] - 11s 68ms/step - loss: 0.6793 - accuracy: 0.5530
Epoch 3/10
157/157 [==============================] - 11s 69ms/step - loss: 0.6754 - accuracy: 0.5544
Epoch 4/10
157/157 [==============================] - 11s 71ms/step - loss: 0.6734 - accuracy: 0.5556
Epoch 5/10
157/157 [==============================] - 10s 63ms/step - loss: 0.6723 - accuracy: 0.5570
Epoch 6/10
157/157 [==============================] - 11s 69ms/step - loss: 0.6714 - accuracy: 0.5583
Epoch 7/10
157/157 [==============================] - 11s 70ms/step - loss: 0.6709 - accuracy: 0.5597
Epoch 8/10
157/157 [==============================] - 11s 69ms/step - loss: 0.6704 - accuracy: 0.5604
Epoch 9/10
157/157 [==============================] - 10s 66ms/step - loss: 0.6696 - accuracy: 0.5577
Epoch 10/10
157/157 [==============================] - 2s 10ms/step - loss: 0.6935